In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
main_data_df = pd.read_csv('main_data_ver2.csv')
main_data_df.shape

(4560736, 24)

In [3]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매


In [48]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:    
        splitted = x.split('-')

        if len(splitted) == 1:
            return x + '-0'
        elif len(splitted) == 3:
            return splitted[1] + '-' + splitted[2]
        else:
            return x

In [49]:
main_data_df['지번주소'] = main_data_df['시군구'] + ' ' + main_data_df['번지'].apply(landnum_modifier)
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1


In [25]:
main_data_df.isna().sum()

시군구                                0
번지                              2435
본번                                 0
부번                                 0
건물명                               12
전월세매매구분                            0
전용면적(㎡)                            0
계약년월                               0
계약일                                0
금액(만원)                             0
월세(만원)                             0
층                                  0
건축년도                               0
도로명                            47380
가격/면적                              0
year                               0
month                              0
시                                  0
구                                  0
target_log_transformed             0
target/area_log_transformed        0
동                                  0
건물종류                               0
trade_type                         0
split_length                    2435
지번주소                            2435
dtype: int64

In [26]:
both_nan_df = main_data_df[(main_data_df['번지'].isna())&(main_data_df['도로명'].isna())]
both_nan_df.shape

(870, 26)

In [27]:
main_data_df.drop(both_nan_df.index, inplace=True)

In [28]:
main_data_df.reset_index(drop=True, inplace=True)

In [29]:
main_data_df.isna().sum()

시군구                                0
번지                              1565
본번                                 0
부번                                 0
건물명                               12
전월세매매구분                            0
전용면적(㎡)                            0
계약년월                               0
계약일                                0
금액(만원)                             0
월세(만원)                             0
층                                  0
건축년도                               0
도로명                            46510
가격/면적                              0
year                               0
month                              0
시                                  0
구                                  0
target_log_transformed             0
target/area_log_transformed        0
동                                  0
건물종류                               0
trade_type                         0
split_length                    1565
지번주소                            1565
dtype: int64

In [30]:
print(main_data_df['시군구'].unique())

['서울특별시 강남구 개포동' '서울특별시 강남구 논현동' '서울특별시 강남구 대치동' '서울특별시 강남구 도곡동'
 '서울특별시 강남구 삼성동' '서울특별시 강남구 세곡동' '서울특별시 강남구 수서동' '서울특별시 강남구 신사동'
 '서울특별시 강남구 압구정동' '서울특별시 강남구 역삼동' '서울특별시 강남구 일원동' '서울특별시 강남구 청담동'
 '서울특별시 강동구 강일동' '서울특별시 강동구 고덕동' '서울특별시 강동구 길동' '서울특별시 강동구 둔촌동'
 '서울특별시 강동구 명일동' '서울특별시 강동구 상일동' '서울특별시 강동구 성내동' '서울특별시 강동구 암사동'
 '서울특별시 강동구 천호동' '서울특별시 강북구 미아동' '서울특별시 강북구 번동' '서울특별시 강북구 수유동'
 '서울특별시 강북구 우이동' '서울특별시 강서구 가양동' '서울특별시 강서구 공항동' '서울특별시 강서구 내발산동'
 '서울특별시 강서구 등촌동' '서울특별시 강서구 마곡동' '서울특별시 강서구 방화동' '서울특별시 강서구 염창동'
 '서울특별시 강서구 화곡동' '서울특별시 관악구 남현동' '서울특별시 관악구 봉천동' '서울특별시 관악구 신림동'
 '서울특별시 광진구 광장동' '서울특별시 광진구 구의동' '서울특별시 광진구 군자동' '서울특별시 광진구 능동'
 '서울특별시 광진구 자양동' '서울특별시 광진구 중곡동' '서울특별시 광진구 화양동' '서울특별시 구로구 가리봉동'
 '서울특별시 구로구 개봉동' '서울특별시 구로구 고척동' '서울특별시 구로구 구로동' '서울특별시 구로구 궁동'
 '서울특별시 구로구 신도림동' '서울특별시 구로구 오류동' '서울특별시 구로구 온수동' '서울특별시 구로구 천왕동'
 '서울특별시 구로구 항동' '서울특별시 금천구 가산동' '서울특별시 금천구 독산동' '서울특별시 금천구 시흥동'
 '서울특별시 노원구 공릉동' '서울특별시 노원구 상계동' '서울특별시 노원구 월계동' '서울특별시 노원구 중계동'
 '서울특별시 노원구 하계동' '서울특별시 도봉

In [31]:
sanggye_df = main_data_df[main_data_df['시군구'].str.contains('상계')]
sanggye_df.shape

(120441, 26)

In [32]:
sanggye_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
375023,서울특별시 노원구 상계동,1321,1321,0,건영,매매,59.76,201304,29,19750,0,15,2003,덕릉로115다길 40,330.488621,2013,4,서울특별시,노원구,9.890909,5.800572,서울특별시 노원구 상계동,아파트,아파트_매매,1.0,서울특별시 노원구 상계동 1321-0
375024,서울특별시 노원구 상계동,1321,1321,0,건영,매매,84.77,201305,2,28200,0,5,2003,덕릉로115다길 40,332.664858,2013,5,서울특별시,노원구,10.247077,5.807136,서울특별시 노원구 상계동,아파트,아파트_매매,1.0,서울특별시 노원구 상계동 1321-0
375025,서울특별시 노원구 상계동,1321,1321,0,건영,매매,59.91,201305,2,20200,0,3,2003,덕릉로115다길 40,337.172425,2013,5,서울특별시,노원구,9.913438,5.820594,서울특별시 노원구 상계동,아파트,아파트_매매,1.0,서울특별시 노원구 상계동 1321-0
375026,서울특별시 노원구 상계동,1321,1321,0,건영,매매,59.76,201305,7,20000,0,12,2003,덕릉로115다길 40,334.672021,2013,5,서울특별시,노원구,9.903488,5.813151,서울특별시 노원구 상계동,아파트,아파트_매매,1.0,서울특별시 노원구 상계동 1321-0
375027,서울특별시 노원구 상계동,1321,1321,0,건영,매매,59.91,201310,5,19000,0,5,2003,덕릉로115다길 40,317.142380,2013,10,서울특별시,노원구,9.852194,5.759351,서울특별시 노원구 상계동,아파트,아파트_매매,1.0,서울특별시 노원구 상계동 1321-0


In [33]:
print(sanggye_df['시군구'].unique())

['서울특별시 노원구 상계동']


In [34]:
length3_df = main_data_df[main_data_df['split_length'] == 3]
length3_df.shape

(325, 26)

In [35]:
length3_df['시군구'].unique()

array(['서울특별시 노원구 상계동', '서울특별시 관악구 봉천동'], dtype=object)

In [36]:
length3_df

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
375504,서울특별시 노원구 상계동,가-52-2,52,2,불암신기 101동,매매,64.58,201305,10,19500,0,4,2002,덕릉로118길 17,301.951068,2013,5,서울특별시,노원구,9.878170,5.710265,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-2
375505,서울특별시 노원구 상계동,가-52-2,52,2,불암신기 101동,매매,64.58,201309,2,20700,0,2,2002,덕릉로118길 17,320.532673,2013,9,서울특별시,노원구,9.937889,5.769984,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-2
375506,서울특별시 노원구 상계동,가-52-2,52,2,불암신기 101동,매매,60.16,201310,3,20000,0,5,2002,덕릉로118길 17,332.446809,2013,10,서울특별시,노원구,9.903488,5.806480,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-2
375507,서울특별시 노원구 상계동,가-52-1,52,1,불암신기 102동,매매,61.77,201301,24,20000,0,4,2002,덕릉로118길 17,323.781771,2013,1,서울특별시,노원구,9.903488,5.780070,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-1
377566,서울특별시 노원구 상계동,가-6040-35,6040,35,코지한주,매매,105.15,201304,6,26000,0,7,2006,덕릉로123길 53,247.265811,2013,4,서울특별시,노원구,10.165852,5.510464,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-6040-35
378328,서울특별시 노원구 상계동,가-52-1,52,1,불암신기 102동,매매,66.15,201405,28,17800,0,1,2002,덕릉로118길 17,269.085412,2014,5,서울특별시,노원구,9.786954,5.595029,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-1
381881,서울특별시 노원구 상계동,가-52-1,52,1,불암신기 102동,매매,62.90,201605,5,22800,0,4,2002,덕릉로118길 17,362.480127,2016,5,서울특별시,노원구,10.034516,5.892970,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-1
385356,서울특별시 노원구 상계동,가-6040-35,6040,35,코지한주,매매,101.41,201606,10,36000,0,7,2006,덕릉로123길 53,354.994576,2016,6,서울특별시,노원구,10.491274,5.872103,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-6040-35
386229,서울특별시 노원구 상계동,가-52-2,52,2,불암신기 101동,매매,64.58,201706,16,25400,0,3,2002,덕릉로118길 17,393.310622,2017,6,서울특별시,노원구,10.142504,5.974600,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-2
386230,서울특별시 노원구 상계동,가-52-1,52,1,불암신기 102동,매매,67.28,201707,7,24500,0,3,2002,덕릉로118길 17,364.149822,2017,7,서울특별시,노원구,10.106428,5.897565,서울특별시 노원구 상계동,아파트,아파트_매매,3.0,서울특별시 노원구 상계동 가-52-1


In [37]:
bongchun_df = length3_df[length3_df['시군구'] == '서울특별시 관악구 봉천동']
bongchun_df.shape

(5, 26)

In [38]:
bongchun_df

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
2840633,서울특별시 관악구 봉천동,가-10-1,10,1,(10-1),매매,49.53,201505,7,15000,0,1,1991,구암길 93-9,302.846760,2015,5,서울특별시,관악구,9.615805,5.713227,서울특별시 관악구 봉천동,다세대연립,다세대연립_매매,3.0,서울특별시 관악구 봉천동 가-10-1
2843867,서울특별시 관악구 봉천동,가-10-1,10,1,(10-1),매매,49.53,201801,11,21500,0,1,1991,구암길 93-9,434.080355,2018,1,서울특별시,관악구,9.975808,6.073230,서울특별시 관악구 봉천동,다세대연립,다세대연립_매매,3.0,서울특별시 관악구 봉천동 가-10-1
2844172,서울특별시 관악구 봉천동,가-11-2,11,2,대성빌라,매매,65.52,201802,11,31000,0,1,1986,구암7길 3,473.137973,2018,2,서울특별시,관악구,10.341742,6.159387,서울특별시 관악구 봉천동,다세대연립,다세대연립_매매,3.0,서울특별시 관악구 봉천동 가-11-2
2847728,서울특별시 관악구 봉천동,가-10-1,10,1,(10-1),매매,49.53,200705,17,13000,0,1,1991,구암길 93-9,262.467192,2007,5,서울특별시,관악구,9.472705,5.570126,서울특별시 관악구 봉천동,다세대연립,다세대연립_매매,3.0,서울특별시 관악구 봉천동 가-10-1
2851484,서울특별시 관악구 봉천동,가-11-2,11,2,대성빌라,매매,65.52,201011,6,29000,0,1,1986,구암7길 3,442.612943,2010,11,서울특별시,관악구,10.275051,6.092696,서울특별시 관악구 봉천동,다세대연립,다세대연립_매매,3.0,서울특별시 관악구 봉천동 가-11-2


In [39]:
samga_df = main_data_df[main_data_df['시군구'].str.contains('3가')]
samga_df.shape

(40667, 26)

In [40]:
samga_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
661924,서울특별시 서대문구 충정로3가,189-8,189,8,미동,매매,56.20,201305,3,14000,0,2,1969,충정로7길 9,249.110320,2013,5,서울특별시,서대문구,9.546813,5.517896,서울특별시 서대문구 충정로,아파트,아파트_매매,2.0,서울특별시 서대문구 충정로3가 189-8
661925,서울특별시 서대문구 충정로3가,466,466,0,우리유앤미,매매,84.73,201301,31,42700,0,17,2008,경기대로 26-7,503.953735,2013,1,서울특별시,서대문구,10.661954,6.222484,서울특별시 서대문구 충정로,아파트,아파트_매매,1.0,서울특별시 서대문구 충정로3가 466-0
661926,서울특별시 서대문구 충정로3가,466,466,0,우리유앤미,매매,84.73,201302,27,43800,0,6,2008,경기대로 26-7,516.936150,2013,2,서울특별시,서대문구,10.687389,6.247919,서울특별시 서대문구 충정로,아파트,아파트_매매,1.0,서울특별시 서대문구 충정로3가 466-0
661927,서울특별시 서대문구 충정로3가,466,466,0,우리유앤미,매매,84.73,201303,3,40000,0,3,2008,경기대로 26-7,472.087808,2013,3,서울특별시,서대문구,10.596635,6.157165,서울특별시 서대문구 충정로,아파트,아파트_매매,1.0,서울특별시 서대문구 충정로3가 466-0
661928,서울특별시 서대문구 충정로3가,466,466,0,우리유앤미,매매,84.73,201303,19,42700,0,6,2008,경기대로 26-7,503.953735,2013,3,서울특별시,서대문구,10.661954,6.222484,서울특별시 서대문구 충정로,아파트,아파트_매매,1.0,서울특별시 서대문구 충정로3가 466-0


In [41]:
main_data_df['번지'].describe()

count     4558301
unique      59500
top            22
freq        16045
Name: 번지, dtype: object

In [44]:
print(main_data_df['부번'].unique())

[   2    1    0 ... 2054 1939 1275]


In [45]:
main_data_df['본번'].max()

6053

In [46]:
main_data_df['부번'].max()

4813

In [47]:
main_data_df['부번'].min()

0

In [50]:
main_data_df.to_csv('main_data_ver2.1.csv', index=False)